# Capstone project
This notebook will be used for the capstone project in which I will use location data to solve a specific problem.

### Part 1 : Creation of the dataframe from the wikipedia page
#### Creating the empty dataframe 

In [1]:
import pandas as pd
import numpy as np

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)
df.head()

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []

### Fill the empty dataframe with the wikipedia table values

In [2]:
from bs4 import BeautifulSoup
import requests

def fillDataframe(wiki_tab_values):
    row_number = 0
    for i in range(0,len(wiki_tab_values),3):
        postal_code = wiki_tab_values[i].string.replace('\n','')
        borough = wiki_tab_values[i+1].string.replace('\n','')
        neighborhood = wiki_tab_values[i+2].string.replace('\n','') 
        if (borough != 'Not assigned'):
            if (neighborhood == 'Not assigned'): 
                neighborhood = borough
            df.loc[row_number] = [postal_code,borough,neighborhood]
            row_number+=1
            
            
respond = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(respond.text, 'html.parser')
tab = soup.find("table",{"class":"wikitable sortable"})
wiki_tab_values = tab.find_all('td')

fillDataframe(wiki_tab_values)
df.head(10)

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
5        M9A         Etobicoke      Islington Avenue, Humber Valley Village
6        M1B       Scarborough                               Malvern, Rouge
7        M3B        North York                                    Don Mills
8        M4B         East York              Parkview Hill, Woodbine Gardens
9        M5B  Downtown Toronto                     Garden District, Ryerson

In [3]:
df.shape

(103, 3)

## Part 2 : Adding Latitude and Longitude columns for each neighborhood
Reading the csv file and creating a dataframe with geospatial data for each postal code.

In [4]:
geodf = pd.read_csv("http://cocl.us/Geospatial_data")
geodf = geodf.rename(columns={'Postal Code': 'PostalCode'})
geodf.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

I will then use a left outer join to merge both table into one, with a common key : PostalCode.
This will result in a new table where we can find the latitude/longitude values for each postal code.

In [5]:
new_df = pd.merge(df, geodf, how='left', on=['PostalCode'])
new_df.head(10)

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
5        M9A         Etobicoke      Islington Avenue, Humber Valley Village   
6        M1B       Scarborough                               Malvern, Rouge   
7        M3B        North York                                    Don Mills   
8        M4B         East York              Parkview Hill, Woodbine Gardens   
9        M5B  Downtown Toronto                     Garden District, Ryerson   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  
5  43.667856 -79.532242  
6  43.806686 -79.194353  
7  43.745906 -79.352188  
8  43.706397 -79.309937  
9  43.657162 -79.378937

# Part 3 : Exploration and clustering or Toronto's neighborhoods

In [6]:
#!conda install -c conda-forge geopy --yes
import folium # map rendering library
from geopy.geocoders import Nominatim

#Keeping the rows with 'Toronto' present in Borough
toronto_df = new_df[new_df['Borough'].str.contains('Toronto')]
toronto_df.head()

PostalCode           Borough                                 Neighborhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  
15  43.651494 -79.375418  
19  43.676357 -79.293031

In [7]:
city = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(toronto_df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
CLIENT_ID = 'AR2LDST4MHLM5KFISIJGCISGINIYHNDZZ5VVP3Q14FMAWLBT' # your Foursquare ID
CLIENT_SECRET = 'DINBAUEQMLLBZQP1V1JSY3V4ZSWFS3FRODTJROZZZAHTRMAZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AR2LDST4MHLM5KFISIJGCISGINIYHNDZZ5VVP3Q14FMAWLBT
CLIENT_SECRET:DINBAUEQMLLBZQP1V1JSY3V4ZSWFS3FRODTJROZZZAHTRMAZ


### Explore Neighborhoods in Toronto
#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [10]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Theater  Theme Restaurant  Toy / Game Store  Trail  \
0             0  ...        0                 0                 0      0   
1             0  ...        0                 0                 0      0   
2             0  ...        0                 0                 0      0   
3             0  ...        0                 0                 0      0   
4             0  ...        0                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Women's Store  
0                      0         0              0  
1                      0         0              0  
2                      0         0              0  
3                      0         0              0  
4                      0         0              0  

[5 rows x 236 columns]

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [11]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.000000   
1       Brockton, Parkdale Village, Exhibition Place     0.000000   
2  Business reply mail Processing Centre, South C...     0.000000   
3  CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                 Central Bay Street     0.014925   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                0.0   0.0000              0.0000        0.0000   
1                0.0   0.0000              0.0000        0.0000   
2                0.0   0.0000              0.0000        0.0000   
3                0.0   0.0625              0.0625        0.0625   
4                0.0   0.0000              0.0000        0.0000   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0           0.000            0.000            0.0000                  0.0   
1           0.000            0.000            0.0000                  0.0   
2           0.000            0.000            0.0000                  0.0   
3           0.125            0.125            0.0625                  0.0   
4           0.000            0.000            0.0000                  0.0   

   ...  Theater  Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0  ...      0.0               0.0               0.0    0.0            0.0   
1  ...      0.0               0.0               0.0    0.0            0.0   
2  ...      0.0               0.0               0.0    0.0            0.0   
3  ...      0.0               0.0               0.0    0.0            0.0   
4  ...      0.0               0.0               0.0    0.0            0.0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                       0.018182               0.0                    0.0   
1                       0.000000               0.0                    0.0   
2                       0.000000               0.0                    0.0   
3                       0.000000               0.0                    0.0   
4                       0.014925               0.0                    0.0   

   Wine Bar  Women's Store  
0       0.0            0.0  
1       0.0            0.0  
2       0.0            0.0  
3       0.0            0.0  
4       0.0            0.0  

[5 rows x 236 columns]

#### Let's print each neighborhood along with the top 5 most common venues

In [12]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1  Seafood Restaurant  0.04
2         Cheese Shop  0.04
3            Beer Bar  0.04
4        Cocktail Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2     Coffee Shop  0.09
3  Breakfast Spot  0.09
4   Grocery Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Fast Food Restaurant  0.06
1         Auto Workshop  0.06
2            Comic Shop  0.06
3           Pizza Place  0.06
4      Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0   Airport Lounge  0.12
1  Airport Service  0.12
2  Harbor / Marina  0.06
3            Plane  0.06
4    Boat or Ferry  0.06


----Central Bay Street----
                venue  freq
0         Coff

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in descending order.

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...            Skate Park   
3  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Farmers Market                Bakery          Cocktail Bar   
1           Coffee Shop        Breakfast Spot             Nightclub   
2           Pizza Place               Brewery         Burrito Place   
3       Airport Service           Coffee Shop       Harbor / Marina   
4                  Café        Sandwich Place    Italian Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Seafood Restaurant           Cheese Shop              Beer Bar   
1             Pet Store                   Bar         Burrito Place   
2            Restaurant        Farmers Market  Fast Food Restaurant   
3                 Plane   Rental Car Location      Sculpture Garden   
4          Burger Joint       Thai Restaurant       Bubble Tea Shop   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Restaurant         Grocery Store                    Pub  
1            Restaurant          Climbing Gym  Performing Arts Venue  
2      Recording Studio               Butcher     Light Rail Station  
3              Boutique                   Bar          Boat or Ferry  
4      Department Store   Japanese Restaurant            Salad Place

## 4. Cluster Neighborhoods
Run _k_-means to cluster the neighborhood into 5 clusters.

In [15]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode           Borough                                 Neighborhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
2   43.654260 -79.360636               0           Coffee Shop   
4   43.662301 -79.389494               0           Coffee Shop   
9   43.657162 -79.378937               0        Clothing Store   
15  43.651494 -79.375418               0           Coffee Shop   
19  43.676357 -79.293031               0     Health Food Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
2                    Pub                  Park                Bakery   
4            Yoga Studio            Hobby Shop            Restaurant   
9            Coffee Shop                  Café        Cosmetics Shop   
15                  Café          Cocktail Bar            Restaurant   
19                   Pub                 Trail               Dog Run   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
2                   Café               Theater        Breakfast Spot   
4         Sandwich Place   Distribution Center                 Diner   
9        Bubble Tea Shop   Japanese Restaurant        Lingerie Store   
15             Gastropub   American Restaurant              Beer Bar   
19          Dessert Shop    Dim Sum Restaurant                 Diner   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
2                Brewery                   Spa             Beer Store  
4     Italian Restaurant              Beer Bar                   Café  
9                  Diner  Fast Food Restaurant                Theater  
15        Clothing Store                   Gym         Cosmetics Shop  
19        Discount Store   Distribution Center          Women's Store

Finally, let's visualize the resulting clusters

In [17]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters